In [ ]:
from sys import path
path.insert(0, '/home/azadoks/git/glosim2/')
path.insert(0, '/home/azadoks/git/structureREST/zocrys/lib/')
from collections import OrderedDict
from libmatch.soap import get_Soaps
import numpy as np
import zoap
import pymongo

In [ ]:
client = pymongo.MongoClient('mongodb://127.0.0.1:27018/')
db = client['structureREST']

icsd_coll = db['icsd']
cod_coll = db['cod']
mpds_coll = db['mpds']

stidy_coll = db['stidy']
matminer_coll = db['matminer']
soap_coll = db['soap']

qp_coll = db['qp']

In [ ]:
for source_collection in [icsd_coll, mpds_coll, cod_coll]:
    for source_doc in source_collection.find()